In [239]:
import pandas as pd 
import numpy as np
import torch
from torch import nn

import torch.nn.functional as F
from torchvision import datasets, transforms


In [241]:
print("I am here!")

my_data = pd.read_csv("train.csv", index_col = 0)

my_data = my_data.drop(columns = ['batch_size_test', 'batch_size_val', 'criterion', 'optimizer', 'batch_size_train'])
y_train = my_data['train_error']
y_val = my_data['val_error']
my_data = my_data.drop(columns = 'train_error')
my_data = my_data.drop(columns = 'val_error')

x = my_data.drop(columns = ['train_loss','val_loss','arch_and_hp','init_params_mu', 'init_params_std', 'init_params_l2'])

col_train = [col for col in x.filter(regex='^train',axis=1).columns]
col_val = [col for col in x.filter(regex='^val',axis=1).columns]
#print(col_train)
#print(col_val)
#print(x)

torch_tensor_train = torch.tensor([x[title].values for title in col_train],dtype=torch.float)
torch_tensor_val = torch.tensor([x[title].values for title in col_val],dtype=torch.float)


torch_tensor_train = torch.t(torch_tensor_train)
torch_tensor_val = torch.t(torch_tensor_val)


print(len(col_train))
print(torch_tensor_train.size())

# ------------- test part -----------------------

test_data = pd.read_csv("test.csv", index_col = 0)
test_data = test_data.drop(columns = ['batch_size_test', 'batch_size_val', 'criterion', 'optimizer', 'batch_size_train'])
test_data = test_data.drop(columns = ['arch_and_hp','init_params_mu', 'init_params_std', 'init_params_l2'])
test_train = test_data[col_train]
test_val = test_data[col_val]

torch_tensor_test_train = torch.tensor([test_train[title].values for title in col_train],dtype=torch.float)
torch_tensor_test_val = torch.tensor([test_val[title].values for title in col_val],dtype=torch.float)

torch_tensor_test_train = torch.t(torch_tensor_test_train)
torch_tensor_test_val = torch.t(torch_tensor_test_val)

#print(test_train)
print(torch_tensor_test_train.size())

I am here!
100
torch.Size([1878, 100])
torch.Size([476, 100])


In [243]:
#we first select a naive model after sorting based on the lowest val_error

#model = torch.nn.Sequential(nn.BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), nn.Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1)) , nn.Tanh() , nn.Dropout2d(p=0.4569261378573387, inplace=False) , nn.Conv2d(32, 27, kernel_size=(3, 3), stride=(1, 1)) , nn.ReLU() , nn.Dropout2d(p=0.022897457910152297, inplace=False) , nn.Conv2d(27, 28, kernel_size=(5, 5), stride=(1, 1)) , nn.LeakyReLU(negative_slope=0.5719652455913541) , nn.MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False) , nn.Flatten() , nn.Linear(in_features=700, out_features=28, bias=True) , nn.LeakyReLU(negative_slope=0.48187737007312437) , nn.Dropout(p=0.3735548245474202, inplace=False) , nn.Linear(in_features=28, out_features=10, bias=True) , nn.BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) )


input_size = 100
hidden_sizes = [60, 30]
output_size = 1

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)


model_train = nn.Sequential(
                      nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      )
#nn.Softmax(dim=0)
# Define the loss
criterion = nn.MSELoss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = torch.optim.SGD(model_train.parameters(), lr=0.003)
epochs = 100
for e in range(epochs):
    running_loss = 0
    for i in range(1878):
        # Training pass
        optimizer.zero_grad()
        
        output = model_train(torch_tensor_train[i])
        #print(output)
        #torch.tensor(y_train[i])
        loss = criterion(output, torch.tensor(y_train[i]))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        
model_val = nn.Sequential(
                      nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      )

# Optimizers require the parameters to optimize and a learning rate
optimizer = torch.optim.SGD(model_val.parameters(), lr=0.003)
epochs = 100
for e in range(epochs):
    running_loss = 0
    for i in range(1878):
        # Training pass
        optimizer.zero_grad()
        
        output = model_val(torch_tensor_val[i])
        #print(output)
        loss = criterion(output, torch.tensor(y_val[i]))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

print("finished")



/Users/Wusongfeng/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


finished


In [245]:

#now we do the prediction

print("I am here!")
res_train,res_val = [],[]

for row in torch_tensor_test_train:
    res_train.append(model_train(row).item())
    
for row in torch_tensor_test_val:
    res_val.append(model_val(row).item())

print(len(res_val))
print(res_train)

I am here!
476
[0.8363585472106934, 0.36869680881500244, 0.5994740724563599, 0.8328453302383423, 0.8060317039489746, 0.5057187080383301, 0.8639028668403625, 0.36450228095054626, 0.7888296842575073, 0.15077286958694458, 0.39467790722846985, 0.38780543208122253, -0.010000735521316528, 0.6006891131401062, 0.23683147132396698, 0.4370037019252777, 0.24581709504127502, 0.4277389645576477, 0.18133270740509033, 0.10450480878353119, 0.021694332361221313, 0.49506059288978577, 0.7593083381652832, 0.04006585478782654, 0.6338014006614685, 0.5114523768424988, 0.8090798854827881, 0.02938300371170044, 0.03021952509880066, 0.284918874502182, 0.861266553401947, 0.20946034789085388, 0.7754078507423401, 0.08231362700462341, 0.02386331558227539, 0.7157660126686096, 0.5392324328422546, 0.7932217121124268, 0.5845357179641724, 0.17960576713085175, 0.6044695377349854, 0.8017895221710205, 0.8576793074607849, 0.8555430173873901, 0.1606748253107071, 0.014012515544891357, 0.1951383352279663, 0.23873388767242432, 0

In [247]:
submit = pd.read_csv("sample_submission.csv", index_col = 0)
res_array = np.zeros(len(res_train)*2)


i = 0
j = 0
while True:
    
    if i == len(res_train):
        break
    
    res_array[j] = res_val[i]
    j += 1
    res_array[j] = res_train[i]
    j += 1
    
    i += 1 

print(res_array)

submit.Predicted = res_array
submit.to_csv("submission.csv")

[ 5.53844213e-01  8.36358547e-01  4.04747903e-01  3.68696809e-01
  5.22167146e-01  5.99474072e-01  6.73149347e-01  8.32845330e-01
  5.95536530e-01  8.06031704e-01  5.40258586e-01  5.05718708e-01
  7.17043161e-01  8.63902867e-01  4.23309684e-01  3.64502281e-01
  6.61685050e-01  7.88829684e-01  3.28574777e-01  1.50772870e-01
  4.71360892e-01  3.94677907e-01  3.98449183e-01  3.87805432e-01
  3.54828089e-01 -1.00007355e-02  4.95136976e-01  6.00689113e-01
  4.24124539e-01  2.36831471e-01  5.04078329e-01  4.37003702e-01
  4.41792965e-01  2.45817095e-01  4.34599578e-01  4.27738965e-01
  3.91934097e-01  1.81332707e-01  4.45642114e-01  1.04504809e-01
  3.84893060e-01  2.16943324e-02  4.12839144e-01  4.95060593e-01
  6.11738145e-01  7.59308338e-01  2.93177247e-01  4.00658548e-02
  5.44147193e-01  6.33801401e-01  4.74901736e-01  5.11452377e-01
  6.59819424e-01  8.09079885e-01  3.27300251e-01  2.93830037e-02
  4.50783670e-01  3.02195251e-02  3.29030216e-01  2.84918875e-01
  7.53093123e-01  8.61266